# Proyecto del Día 17: Dashboard de Terremotos a Nivel Mundial

Bienvenido al proyecto del día de hoy, donde vas a crear un **dashboard interactivo utilizando Plotly y Dash**. En este proyecto, trabajarás con datos reales de terremotos, que podrás descargar de los archivos adjuntos de esta lección.

Con esa información vas a desarrollar un conjunto de visualizaciones interactivas que proporcionen una comprensión profunda de los patrones y tendencias de los terremotos. Este ejercicio te permitirá aplicar tus conocimientos adquiridos sobre visualización de datos y desarrollo de dashboards con herramientas de Python.

El dataset contiene información detallada sobre terremotos, incluyendo:
+ fecha y hora del evento
+ magnitud
+ profundidad
+ país en el que ocurrió.

Utilizarás esta información para crear varios gráficos interactivos que te ayudarán a explorar y presentar los datos de manera efectiva.

### Consigna

Tu tarea es desarrollar un dashboard interactivo utilizando **Plotly** y **Dash** para visualizar los datos de terremotos proporcionados en el archivo `earthquake data.csv`. 

Asegúrate de que el dashboard tenga un diseño limpio y organizado, con títulos claros y descripciones para cada gráfico.

### Pasos para Completar el Proyecto

1. **Cargar y Preparar los Datos**: Carga el archivo `earthquake data.csv` y realiza las transformaciones necesarias, como convertir la columna de fecha y hora al formato datetime.

2. **Creación de Gráficos**: Crea los siguientes gráficos usando plotly:
    + **Mapa Coroplético**: Un mapa coroplético que muestre la densidad de terremotos por región.
    + **Gráfico de Barras**: Un gráfico de barras que muestre el número de terremotos por magnitud.
    + **Gráfico de Líneas**: Un gráfico de líneas que muestre la tendencia de la magnitud de los terremotos a lo largo del tiempo.
    + **Gráfico de Dispersión**: Un gráfico de dispersión que muestre la relación entre la magnitud de los terremotos y su profundidad.
    + **Gráfico 3D**: Un gráfico 3D que visualice la relación entre magnitud, profundidad y tiempo.

3. **Diseño de la Aplicación**: Usando Dash, crea la aplicación con el Layout del Dashboard, en el que deben desplegarse todos los gráficos que has preparado. Finalmente ejecuta la aplicación.


Ten en cuenta que la información entra por los ojos, por lo que sería bueno que agregues parámetros de formato a cada gráfico para mejorar su apariencia y usabilidad. Piensa en colores, tamaños de marcadores, etiquetas de texto y cualquier otra personalización que haga los gráficos más informativos y atractivos. No te limites a lo enseñado en las lecciones y atrévete a explorar parámetros adicionales.

**¡Buena suerte y diviértete creando tu dashboard interactivo de terremotos!**

## 1. Cargar y Preparar los Datos

In [3]:
from typing import Any

import numpy as np
import pandas as pd
import cufflinks as cf
import plotly.io as pio

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import dash
from dash import dcc, html

In [4]:
df: pd.DataFrame = pd.read_csv('./data/earthquake+data.csv')
df.head()

,Date & Time,Latitude,Longitude,Depth,Magnitude,Lands,Country
0,11/21/2023 16:36,38243.0,38808.0,7,1.0,EASTERN,TURKEY
1,11/21/2023 15:59,39134.0,29026.0,10,1.0,WESTERN,TURKEY
2,11/21/2023 13:00,39636.0,30104.0,7,1.0,WESTERN,TURKEY
3,11/20/2023 18:43,47368.0,6918.0,5,1.0,SWITZERLAND,SWITZERLAND
4,11/19/2023 10:48,37175.0,36892.0,7,1.0,CENTRAL,TURKEY


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52767 entries, 0 to 52766
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date & Time  52767 non-null  object 
 1   Latitude     52767 non-null  float64
 2   Longitude    52767 non-null  float64
 3   Depth        52767 non-null  int64  
 4   Magnitude    52767 non-null  float64
 5   Lands        52767 non-null  object 
 6   Country      52767 non-null  object 
dtypes: float64(3), int64(1), object(3)
memory usage: 2.8+ MB


In [6]:
type(df['Date & Time'][0])

str

In [7]:
df['Date & Time'] = pd.to_datetime(
    df['Date & Time'],
    format='%m/%d/%Y %H:%M',
)
print(df['Date & Time'][0])
print(type(df['Date & Time'][0]))

2023-11-21 16:36:00
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


## 2. Creación de Gráficos

### Mapa Coroplético

Un mapa coroplético que muestre la **densidad de terremotos por región**.

**NOTA DE RECORDATORIO / APRENDIZAJE/  DISCLAIMER:** Mi primer pensamiento natural (e intento) de _n00000b_ fue el de utilizar directamente los "países" que se encuentran en `df['Country']`, de manera similar al ejercicio realizado en la [clase correspondiente](../maps.ipynb). Sin embargo, el resultado no fue el esperado, ya que no se generan las geometrías de los países (y, por ende, tampoco las coloraciones para indicar la densidad de terremotos).

Investigando más a fondo, el asunto tiene mucho que ver con [cómo se generan dichas geometrías](https://plotly.com/python/choropleth-maps/#introduction-main-parameters-for-choropleth-outline-maps). Dado que no hay un archivo `GeoJSON` asociado al dataset de trabajo (opción A), no queda de otra que usar una de las geometrías incluidas en `plotly` (opción B).

Al respecto, la [documentación](https://plotly.com/python/choropleth-maps/#using-builtin-country-and-state-geometries) indica lo siguiente:

> To use the built-in countries geometry, provide locations as three-letter ISO country codes.

Como se ha visto en las celdas exploratorias de `df.head()` y `df.info()` (y comparando con el dataset incluido en `px.data.gapminder()` trabajado en la [clase correspondiente](../maps.ipynb)), nuestro dataset **no contiene columnas ISO**.

La estrategia de generar una nueva columna _ad hoc_ sería relativamente fácil, de no ser porque si vemos directamente el listado de "países" en nuestro `df`...

In [8]:
df['Country'].unique()

array(['TURKEY', 'SWITZERLAND', 'SLOVENIA', 'FRANCE', 'ITALY', 'REGION',
       'GERMANY', 'LIECHTENSTEIN', 'CROATIA', 'AUSTRIA', 'PORTUGAL',
       'GREECE', 'HAWAII', 'TEXAS', 'GIBRALTAR', 'CALIFORNIA', 'MEXICO',
       'REG', 'SEA', 'AUSTRALIA', 'IDAHO', 'MONTANA', 'PYRENEES', 'SPAIN',
       'UTAH', 'IS.', 'CANADA', 'NEVADA', 'TENNESSEE', 'RICO', 'CALIF.',
       'ALASKA', 'ALBANIA', 'REPUBLIC', 'ARKANSAS', 'AZERBAIJAN',
       'ISLANDS', 'WASHINGTON', 'SYRIA', 'KYRGYZSTAN', 'OKLAHOMA',
       'WYOMING', 'ARMENIA', 'OREGON', 'PENINSULA', 'OHIO', 'MOROCCO',
       'BULGARIA', 'ROMANIA', 'NETHERLANDS', 'UKRAINE', 'KENTUCKY', 'USA',
       'MISSOURI', 'HUNGARY', 'CAROLINA', 'COLORADO', 'MACEDONIA',
       'ALGERIA', 'YORK', 'MONTENEGRO', 'PHILIPPINES', 'SALVADOR',
       'CHILE', 'ARGENTINA', 'RICA', 'INDONESIA', 'BOLIVIA', 'N.Z.',
       'NICARAGUA', 'SUMATRA', 'ZEALAND', "(SAK'ART'VELO)", 'AFRICA',
       'NORWAY', 'CAN.', 'PANAMA', 'CHINA', 'POLAND', 'INDIA', 'NEPAL',
       'TAJIK

...nos podremos percatar rápidamente de que, aparte de los nombres de países, vemos también nombres de subdivisiones nacionales como estados de los EE.UU. (`'HAWAII'` o `'TEXAS'`), abreviaturas sin clarificación como `'IS.'` o `'P.N.G.'`, nombres incompletos y sin contexto como `'REPUBLIC'` o `'GULF'`, nombres de continentes como `'AMERICA'` o `'AFRICA'`, e incluso autónimos de países como `"(SAK'ART'VELO)"` (para referirse al país caucásico de [Georgia](https://es.wikipedia.org/wiki/Georgia), seguramente para distinguirlo del estado homónimo de los EE.UU.).

Afortunadamente, la misma [documentación nos entrega una solución funcional para este caso](https://plotly.com/python/density-heatmaps/), con la cual utilizar las columnas `df['Latitude']` y `df['Longitude']`.

In [18]:
earthquakes_by_region: go.Figure = px.density_map(
    df,
    lat=df['Latitude'],
    lon=df['Longitude'],
    z=df['Magnitude'],
    center=dict(lat=0, lon=0),
    radius=2,
    zoom=0,
    title='Densidad de terremotos por región',
    map_style='carto-darkmatter'
)

earthquakes_by_region.show()

### Gráfico de Barras

Un gráfico de barras que muestre el **número de terremotos por magnitud**.

### Gráfico de Líneas

Un gráfico de líneas que muestre la tendencia de la **magnitud de los terremotos a lo largo del tiempo**.

### Gráfico de Dispersión

Un gráfico de dispersión que muestre la **relación entre la magnitud de los terremotos y su profundidad**.

### Gráfico 3D

Un gráfico 3D que visualice la **relación entre magnitud, profundidad y tiempo**.